In [3]:
import pydap
import pydap.client
import pydap.parsers.dmr
import xml
import numpy
import sys

# Dap2

In [4]:
ds = pydap.client.open_dods_file('data/coads_climatology.nc.dods')
ds = pydap.client.open_dods_file('data/ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp.dods')

In [5]:
ds['/gt1r/bckgrd_atlas/delta_time']

<BaseType with data array([31197611.26648983, 31197611.27148983, 31197611.27648983, ...,
       31198058.70150019, 31198058.70650019, 31198058.71150019])>

# Dap 4

In [10]:
file_path = 'data/ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp.dap'
file_path = 'data/20220102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1_subset.dap'
file_path = 'data/20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.dap'
file_path = 'data/coads_climatology.nc.dap'
file_path = 'data/MY1DQND1.sst.ADD2005001.040.2006011070802.hdf.dap'

In [ ]:
dataset = pydap.client.open_dap_file(file_path)
dataset

# Read Sizes, then DMR, then Data

## First 4 bytes are the CRLF and length of the DMR

In [11]:
with open(file_path, "rb") as f:  
    clfr = f.read(2)
    dmr_len = numpy.frombuffer(f.read(2), dtype='>u2')[0]
dmr_len

30829

In [12]:
dmr_len

30829

In [13]:
dmr_len = pydap.client.get_dmr_length(file_path)
dmr_len

9484

## Read the DMR

In [14]:
with open(file_path, "rb") as f:    
    clfr = f.read(4)
    dmr = f.read(dmr_len)
dmr = dmr.decode('ascii')

In [16]:
dataset = pydap.client.open_dmr_file(file_path)
dataset

<DatasetType with children 'sst_qual_b', 'Number_of_records', 'Number_of_samples_per_record', 'Latitude', 'Longitude'>

## Convert DMR to dataset

## Read Data

In [17]:
def decode_chunktype(chunk_type):      
    encoding = '{0:03b}'.format(chunk_type)    
    if sys.byteorder == 'little':
        encoding = encoding[::-1]       
    last_chunk = bool(int(encoding[0]))    
    error = bool(int(encoding[1]))    
    endian = {'0': '>', '1': '<'}[encoding[2]]
    return last_chunk, error, endian
    
def get_length(variable):
    dtype = variable.dtype        
    shape = variable.shape
    print(shape)
    length = int(numpy.array(shape).prod())
    itemsize = numpy.dtype(dtype).itemsize
    count = length*itemsize        
    return count
    
def decode_variable(buffer, start, stop, variable, endian):        
    dtype = variable.dtype
    dtype = dtype.newbyteorder(endian)    
    data = numpy.frombuffer(buffer[start:stop], dtype=dtype).astype(dtype)
    data = data.reshape(variable.shape)               
    return data

def get_endianness(f):
    chunk_header = f.peek()[0:4]    
    chunk_header = numpy.frombuffer(chunk_header, dtype='>u4')[0]    
    chunk_type = ((chunk_header >> 24) & 0xff)
    last, error, endian = decode_chunktype(chunk_type)         
    return endian
    
    
with open(file_path, "rb") as f:    
    f.seek(dmr_len)
    clfr = f.read(4)
    
    last = False
    buffer = bytearray()
    get_endianness(f)
    while last == False:
        chunk_header = f.read(4)
        chunk_header = numpy.frombuffer(chunk_header, dtype='>u4')[0]
        chunk_size = (chunk_header & 0x00ffffff)
        chunk_type = ((chunk_header >> 24) & 0xff)
        last, error, endian = decode_chunktype(chunk_type) 
        buffer.extend(f.read(chunk_size))  

In [18]:
start = 0
d = []

checksum_dtype = numpy.dtype(endian + 'u4')
for variable_name in dataset:    
    variable = dataset[variable_name]        
    length = int(get_length(variable))
    stop = start+length
    data = decode_variable(buffer, start=start, stop=stop, variable=variable, endian=endian)
    d.append(data)
    checksum = numpy.frombuffer(buffer[stop:stop+4], dtype=checksum_dtype).byteswap('=')
    start = stop + 4

(180, 360)


ValueError: cannot reshape array of size 0 into shape (180,360)

# Read all and split

In [27]:
with open(file_path, "rt", buffering=1, encoding='ascii', newline='\n', errors='ignore') as f:
    f.seek(4)
    dap = f.read()
    
dmr = dap.split('</Dataset>')[0]
data = dap.split('</Dataset>')[1]
dmr = dmr +'</Dataset>'

dom = xml.dom.minidom.parseString(dmr.strip())

In [28]:
with open(file_path, "rt", buffering=1, encoding='ascii', newline='\n', errors='ignore') as f:
    
    dmr = f.read(len(dmr))
dmr

'\x04\x00\x1c<?xml version="1.0" encoding="ISO-8859-1"?>\n<Dataset xmlns="http://xml.opendap.org/ns/DAP/4.0#" xml:base="http://test.opendap.org:8080/opendap/dmrpp_test_files/ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp" dapVersion="4.0" dmrVersion="1.0" name="ATL03_20181228015957_13810110_003_01.h5">\n    <Group name="gt1r">\n        <Attribute name="Description" type="String">\n            <Value>Each group contains the segments for one Ground Track. As ICESat-2 orbits the earth, sequential transmit pulses illuminate six ground tracks on the surface of the earth.  The track width is approximately 14m.  Each ground track is numbered, according to the laser spot number that generates a given ground track.  Ground tracks are numbered from the left to the right in the direction of spacecraft travel as: 1L, 1R in the left-most pair of beams; 2L, 2R for the center pair of beams; and 3L, 3R for the right-most pair of beams.</Value>\n        </Attribute>\n        <Attribute name="atlas_

In [29]:
dmr = ''
with open(file_path, "rb") as f:    
    dap = f.read()
    
# CRLF: ASCII 13 + ASCII 10
pos = len(dmr)+3

with open(file_path, "rb") as f:    
    f.seek(pos)
    
    chunk = numpy.frombuffer(f.read(4), dtype='<i4')
    data = numpy.frombuffer(f.read(8), dtype='<d').astype('<d')
    
    r = numpy.frombuffer(f.read(8), dtype='<i8').astype('<i8')
    print(r)
    
pos

[2463519784734387823]


3

In [30]:
dmr = dap.split(b'</Dataset>')[0] + b'</Dataset>'
len(dmr)
#7504

7330

# Live

In [34]:
url = 'http://test.opendap.org/opendap/data/nc/coads_climatology.nc.dap'
r = pydap.net.GET(url)
crlf = r.body.find('\r\n'.encode())
dmr = r.body[4:crlf]
dataset = pydap.parsers.dmr.dmr_to_dataset(dmr.decode())

total_nelems = 1
bytes_per_item = 2
for var in dataset: 
    for var_dim_size in dataset[var].shape:
        print(f"var_dim_size {var_dim_size}")
        total_nelems *= int(var_dim_size)
        total_data_size = total_nelems * bytes_per_item
        
    #dataset[var].data.extend(total_nelems)
    print("total_nelems " + str(total_nelems) + " total_data_size " + str(total_data_size))

var_dim_size 180
total_nelems 180 total_data_size 360
var_dim_size 90
total_nelems 16200 total_data_size 32400
var_dim_size 12
total_nelems 194400 total_data_size 388800
var_dim_size 12
var_dim_size 90
var_dim_size 180
total_nelems 37791360000 total_data_size 75582720000
var_dim_size 12
var_dim_size 90
var_dim_size 180
total_nelems 7346640384000000 total_data_size 14693280768000000
var_dim_size 12
var_dim_size 90
var_dim_size 180
total_nelems 1428186890649600000000 total_data_size 2856373781299200000000
var_dim_size 12
var_dim_size 90
var_dim_size 180
total_nelems 277639531542282240000000000 total_data_size 555279063084564480000000000


# Downloading the constrained dataset

In [73]:
subset = pydap.lib.quote('/SST[0:1:1][1:1:2][0:1:3]')
dataset = 'http://test.opendap.org/opendap/data/nc/coads_climatology.nc.dap'
url = f'{dataset}?dap4.ce={subset}'
ret = requests.get(url)
print(url)

http://test.opendap.org/opendap/data/nc/coads_climatology.nc.dap?dap4.ce=/SST%5B0%3A1%3A1%5D%5B1%3A1%3A2%5D%5B0%3A1%3A3%5D


# Splitting and parsing the DMR and Data

In [76]:
dmr_len = ret.content[0:4]
dmr_len = numpy.frombuffer(dmr_len, dtype='>i')

dmr = ret.content.split(b'</Dataset>')[0][4:] + b'</Dataset>'
dmr_dom = xml.dom.minidom.parseString(dmr)
data = ret.content.split(b'</Dataset>\n\r\n')[1]

# Getting the element size from the DOM

In [79]:
dmr_dom.getElementsByTagName('Dim')[0].getAttribute('size')

'2'

# Pulling out the chunk information

In [80]:
chunk = numpy.frombuffer(data[0:4], dtype='>i4')
chunk_size = (chunk & 0x00ffffff)[0]
chunk_code_int = ((chunk >> 24) & 0xff)[0]

print(chunk_code_int, chunk_size)

4 68


## Chunk Type Encoding

|Bit #|	Value of 0	|Value of 1|
| :--- | :---|:--- | 
|0	|A data containing chunk|	The last data chunk|
|1	|The current chunk is not an error chunk.|The current chunk is an "error chunk" and contains an error message|
|2	|The data in this response is encoded using Big-Endian (i.e. network byte order)	|The data in this response is encoded using Little-Endian|


In [81]:
chunk_code = bin(chunk_code_int)
chunk_code

'0b100'

# Getting values

In [82]:
values = numpy.frombuffer(data[4:chunk_size], dtype='<f').astype('<f')
values

array([-1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34,
       -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34,
       -1.e+34, -1.e+34], dtype=float32)

# Getting checksum

In [83]:
checksum = numpy.frombuffer(data[12:20], dtype='<i8').astype('<i8')
checksum

array([-579129404597828385])

# Experiments

In [31]:
import struct


d, = struct.unpack('>i', b'\x04\x00\x03\xf1')
d

67109873

In [32]:
'\x04\x00\x00'
'\x05\x00\x00\x00'

'\x05\x00\x00\x00'

In [33]:
DAP2_ARRAY_LENGTH_NUMPY_TYPE = '>I'